## 라이브러리 임포트

In [11]:
import tkinter as tk
from PIL import Image, ImageTk
import os

## 조 이름, 조원 정보 입력 기능

- 주의 사항
  - 팀 정보 저장 형태 : `[{'team_number': 1, 'team_member': '다람쥐 고양이 레고 '}, {'team_number': 2, 'team_member': '최현정 전혜준 김경환 '}]`

- 개선할 점
  - 유효성 검사 기능 추후 검토
  - 사용자 입력시 왼쪽상단에 작은 박스로 입력 문자가 표시됨 왤까?

### 전역변수 설정


In [12]:
# 전역 변수 설정
team_info = []
input_texts = ["" for _ in range(7)]
max_teams = 7
img_objects = []
image_id = None
image_ids = []
input_positions = [100]
text_ids = [None for _ in range(7)]
cursor_id = None
edit_button_ids = [None for _ in range(7)]
edit_mode = [False for _ in range(7)]
input_spacing = 75
active_input = None
input_image_files = [f"../img/icons/input{i}.png" for i in range(2, 8)]

### 텍스트 입력 및 표시 관련 함수

- 키보드 입력을 처리하는 함수

- 매개변수:
    - event : 키보드 이벤트 객체

- 설명:
현재 활성화된 입력칸에 텍스트를 추가하거나 삭제하고, 
텍스트 디스플레이와 커서 위치를 업데이트합니다.

In [13]:
# 텍스트 입력 및 표시 관련 함수
def handle_input(event):
    global active_input
    if active_input is not None:
        if event.keysym == 'BackSpace':
            input_texts[active_input] = input_texts[active_input][:-1]
        elif event.keysym == 'space':
            input_texts[active_input] += ' '
        # 사용자가 입력한 문자를 처리하는 역할
        elif len(event.char) == 1:
            input_texts[active_input] += event.char
        update_text_display(active_input)
        update_cursor_position(active_input)

### 입력칸 텍스트 표시 함수
    
- 특정 입력칸의 텍스트 디스플레이를 업데이트하는 함수

- 매개변수:
    - index : 업데이트할 입력칸의 인덱스

- 설명:
기존 텍스트를 삭제하고 새로운 텍스트를 캔버스에 표시합니다.

In [14]:
def update_text_display(index):
    if text_ids[index]:
        canvas.delete(text_ids[index])
    y_position = input_positions[index]
    text_ids[index] = canvas.create_text(180, y_position + 25, text=input_texts[index], anchor='w', font=('Arial', 12))

### 입력칸 내 커서 표시 함수

- 매개변수:
    - index : 커서를 표시할 입력칸의 인덱스

- 설명:
현재 텍스트의 끝에 커서를 표시하고 깜빡임 효과를 시작합니다.

In [15]:
def update_cursor_position(index):
    global cursor_id
    if cursor_id:
        canvas.delete(cursor_id)
    y_position = input_positions[index]
    text_width = canvas.bbox(text_ids[index])[2] - canvas.bbox(text_ids[index])[0] if text_ids[index] else 0
    cursor_id = canvas.create_line(180 + text_width, y_position + 10, 180 + text_width, y_position + 40, fill='black')
    blink_cursor()

### 커서 깜빡임 효과

- 설명:
    600ms 간격으로 커서를 보이게 하거나 숨깁니다.


In [24]:
def blink_cursor():
    global cursor_id
    if cursor_id:
        current_state = canvas.itemcget(cursor_id, 'state')
        new_state = 'hidden' if current_state == 'normal' else 'normal'
        canvas.itemconfigure(cursor_id, state=new_state)
    if active_input is not None:
        canvas.after(700, blink_cursor)

### 특정 입력칸 활성화

- 특정 입력칸을 활성화하는 함수
    
- 매개변수:
    - index : 활성화할 입력칸의 인덱스

- 설명:
지정된 입력칸을 활성 상태로 만들고 커서를 표시합니다.

In [17]:
def activate_input(index):
    global active_input
    active_input = index
    canvas.focus_set()
    update_cursor_position(index)

### 입력칸 추가 기능

- 새로운 입력칸을 추가하는 함수

- 설명:
최대 팀 수에 도달하지 않았다면 새 입력칸을 추가하고 관련 이미지를 로드합니다.

In [18]:
def add_input_field():
    if len(input_positions) < max_teams:
        new_index = len(input_positions)
        y_position = input_positions[0] + new_index * input_spacing
        input_positions.append(y_position)
        
        try:
            if new_index > 0:
                img_entry = tk.PhotoImage(file=input_image_files[new_index - 1])
                img_objects.append(img_entry)
                image_id = canvas.create_image(635, y_position + 25, image=img_entry)
                image_ids.append(image_id)
            
            input_texts[new_index] = ""
            update_text_display(new_index)
            
            print(f"새 입력칸 추가됨: {new_index + 1}번째")
        except tk.TclError as e:
            print(f"이미지 로딩 오류: {e}")
            print(f"시도한 경로: {input_image_files[new_index - 1]}")

### 수정하기 버튼 기능

- display_edit_button
    -  수정하기 버튼을 표시하는 함수
        
    - 매개변수:
        - index : 수정하기 버튼을 표시할 입력칸의 인덱스

    - 설명:
    지정된 입력칸 옆에 수정하기 버튼 이미지를 로드하고 표시합니다.

- get_button_position
    - 수정하기 버튼의 위치를 계산하는 함수
    
    - 매개변수:
        - index : 버튼 위치를 계산할 입력칸의 인덱스

    - 반환값:
    (x, y) : 버튼의 x, y 좌표

    - 설명:
    첫 번째 버튼은 고정 위치에, 나머지는 입력칸에 맞춰 위치를 계산합니다.


In [19]:
def display_edit_button(index):
    try:
        edit_img = tk.PhotoImage(file='../img/icons/edit_button.png')
        img_objects.append(edit_img)
        button_x, button_y = get_button_position(index)
        edit_button_ids[index] = canvas.create_image(button_x, button_y, image=edit_img)
    except tk.TclError as e:
        print(f"이미지 로딩 오류: {e}")
        print(f"시도한 경로: ../img/icons/edit_button.png")

def get_button_position(index):
    if index == 0:
        return 1070, 130
    else:
        y_position = input_positions[index]
        return 1070, y_position + 25

### 이벤트 처리

- 마우스 클릭 이벤트를 처리하는 함수
    
- 매개변수:
    - event : 마우스 클릭 이벤트 객체

- 설명:
클릭된 위치에 따라 입력칸 활성화, 추가 버튼 처리, 수정 버튼 처리를 수행합니다.


In [20]:
def handle_page(event):
    global active_input, cursor_id

    if cursor_id:
        canvas.delete(cursor_id)
        cursor_id = None

    for i, y_position in enumerate(input_positions):
        if 160 < event.x < 1000 and y_position < event.y < y_position + 50:
            print(f"입력칸 {i+1} 클릭됨")
            activate_input(i)
            return
    
    for i, y_position in enumerate(input_positions):
        if 1150 < event.x < 1250 and y_position < event.y < y_position + 50:
            if i >= len(team_info):
                handle_add_button(i)
            else:
                handle_edit_button(i)
            return

    for i in range(len(team_info)):
        button_x, button_y = get_button_position(i)
        if button_x - 40 < event.x < button_x + 40 and button_y - 20 < event.y < button_y + 20:
            handle_edit_button(i)
            return

    active_input = None

### 버튼 클릭 처리

- handle_add_button
    - 추가 버튼 클릭을 처리하는 함수
    
    - 매개변수:
        - index : 클릭된 추가 버튼의 인덱스

    - 설명:
    입력된 팀 정보를 저장하고, 새 입력칸을 추가하며, 수정 버튼을 표시합니다.

- handle_edit_button
    - 수정하기 버튼 클릭을 처리하는 함수
    
    - 매개변수:
        - index : 클릭된 수정하기 버튼의 인덱스

    - 설명:
    입력된 새로운 팀 정보로 기존 정보를 업데이트합니다.


In [21]:
def handle_add_button(index):
    print(f"추가 버튼 클릭됨 (입력칸 {index+1})")
    if input_texts[index]:
        team_number = len(team_info) + 1
        team_member = input_texts[index]
        team_info.append({'team_number': team_number, 'team_member': team_member})
        print(f"저장된 팀 정보: {team_info}")
        if len(input_positions) < max_teams:
            add_input_field()
        display_edit_button(index)

def handle_edit_button(index):
    print(f"수정하기 버튼 클릭됨 (입력칸 {index+1})")
    if input_texts[index]:
        team_info[index]['team_member'] = input_texts[index]
        print(f"팀 정보 업데이트: {team_info}")
    else:
        print("입력된 내용이 없어 업데이트하지 않았습니다.")


### 메인 애플리케이션 설정

In [25]:
root = tk.Tk()
root.title("누가 누가 꼴찌할까 - 팀 정보 입력")

canvas = tk.Canvas(root, width=1280, height=720)
canvas.pack()

print("현재 작업 디렉토리:", os.getcwd())

try:
    img = tk.PhotoImage(file='../img/background_pages/team_input.png')
    img_objects.append(img)
    image_id = canvas.create_image(640, 360, image=img)
    image_ids.append(image_id)
except tk.TclError as e:
    print(f"이미지 로딩 오류: {e}")
    print(f"시도한 경로: ../img/background_pages/team_input.png")
    img = tk.PhotoImage(width=1, height=1)

canvas.bind("<Button-1>", handle_page)
root.bind("<Key>", handle_input)

root.mainloop()

현재 작업 디렉토리: c:\Users\PC\Desktop\게임 개발\who-will-be-last\최현정


In [23]:
# # 전역 변수 설정
# team_info = []
# input_texts = ["" for _ in range(7)]
# max_teams = 7
# img_objects = []
# image_id = None
# image_ids = []
# input_positions = [100]
# text_ids = [None for _ in range(7)]
# cursor_id = None
# edit_button_ids = [None for _ in range(7)]
# edit_mode = [False for _ in range(7)]
# input_spacing = 75
# active_input = None
# input_image_files = [f"../img/icons/input{i}.png" for i in range(2, 8)]

# # 텍스트 입력 및 표시 관련 함수
# def handle_input(event):
#     global active_input
#     if active_input is not None:
#         if event.keysym == 'BackSpace':
#             input_texts[active_input] = input_texts[active_input][:-1]
#         elif event.keysym == 'space':
#             input_texts[active_input] += ' '
#         elif len(event.char) == 1:
#             input_texts[active_input] += event.char
#         update_text_display(active_input)
#         update_cursor_position(active_input)

# def update_text_display(index):
#     if text_ids[index]:
#         canvas.delete(text_ids[index])
#     y_position = input_positions[index]
#     text_ids[index] = canvas.create_text(180, y_position + 25, text=input_texts[index], anchor='w', font=('Arial', 12))

# def update_cursor_position(index):
#     global cursor_id
#     if cursor_id:
#         canvas.delete(cursor_id)
#     y_position = input_positions[index]
#     text_width = canvas.bbox(text_ids[index])[2] - canvas.bbox(text_ids[index])[0] if text_ids[index] else 0
#     cursor_id = canvas.create_line(180 + text_width, y_position + 10, 180 + text_width, y_position + 40, fill='black')
#     blink_cursor()

# def blink_cursor():
#     global cursor_id
#     if cursor_id:
#         current_state = canvas.itemcget(cursor_id, 'state')
#         new_state = 'hidden' if current_state == 'normal' else 'normal'
#         canvas.itemconfigure(cursor_id, state=new_state)
#     if active_input is not None:
#         canvas.after(600, blink_cursor)

# def activate_input(index):
#     global active_input
#     active_input = index
#     canvas.focus_set()
#     update_cursor_position(index)

# # 입력칸 및 버튼 관리 함수
# def add_input_field():
#     if len(input_positions) < max_teams:
#         new_index = len(input_positions)
#         y_position = input_positions[0] + new_index * input_spacing
#         input_positions.append(y_position)
        
#         try:
#             if new_index > 0:
#                 img_entry = tk.PhotoImage(file=input_image_files[new_index - 1])
#                 img_objects.append(img_entry)
#                 image_id = canvas.create_image(635, y_position + 25, image=img_entry)
#                 image_ids.append(image_id)
            
#             input_texts[new_index] = ""
#             update_text_display(new_index)
            
#             print(f"새 입력칸 추가됨: {new_index + 1}번째")
#         except tk.TclError as e:
#             print(f"이미지 로딩 오류: {e}")
#             print(f"시도한 경로: {input_image_files[new_index - 1]}")

# def display_edit_button(index):
#     try:
#         edit_img = tk.PhotoImage(file='../img/icons/edit_button.png')
#         img_objects.append(edit_img)
#         button_x, button_y = get_button_position(index)
#         edit_button_ids[index] = canvas.create_image(button_x, button_y, image=edit_img)
#     except tk.TclError as e:
#         print(f"이미지 로딩 오류: {e}")
#         print(f"시도한 경로: ../img/icons/edit_button.png")

# def get_button_position(index):
#     if index == 0:
#         return 1070, 130
#     else:
#         y_position = input_positions[index]
#         return 1070, y_position + 25

# # 이벤트 처리 함수
# def handle_page(event):
#     global active_input, cursor_id

#     if cursor_id:
#         canvas.delete(cursor_id)
#         cursor_id = None

#     for i, y_position in enumerate(input_positions):
#         if 160 < event.x < 1000 and y_position < event.y < y_position + 50:
#             print(f"입력칸 {i+1} 클릭됨")
#             activate_input(i)
#             return
    
#     for i, y_position in enumerate(input_positions):
#         if 1150 < event.x < 1250 and y_position < event.y < y_position + 50:
#             if i >= len(team_info):
#                 handle_add_button(i)
#             else:
#                 handle_edit_button(i)
#             return

#     for i in range(len(team_info)):
#         button_x, button_y = get_button_position(i)
#         if button_x - 40 < event.x < button_x + 40 and button_y - 20 < event.y < button_y + 20:
#             handle_edit_button(i)
#             return

#     active_input = None

# def handle_add_button(index):
#     print(f"추가 버튼 클릭됨 (입력칸 {index+1})")
#     if input_texts[index]:
#         team_number = len(team_info) + 1
#         team_member = input_texts[index]
#         team_info.append({'team_number': team_number, 'team_member': team_member})
#         print(f"저장된 팀 정보: {team_info}")
#         if len(input_positions) < max_teams:
#             add_input_field()
#         display_edit_button(index)

# def handle_edit_button(index):
#     print(f"수정하기 버튼 클릭됨 (입력칸 {index+1})")
#     if input_texts[index]:
#         team_info[index]['team_member'] = input_texts[index]
#         print(f"팀 정보 업데이트: {team_info}")
#     else:
#         print("입력된 내용이 없어 업데이트하지 않았습니다.")

# # 메인 애플리케이션 설정
# root = tk.Tk()
# root.title("누가 누가 꼴찌할까 - 팀 정보 입력")

# canvas = tk.Canvas(root, width=1280, height=720)
# canvas.pack()

# print("현재 작업 디렉토리:", os.getcwd())

# try:
#     img = tk.PhotoImage(file='../img/background_pages/team_input.png')
#     img_objects.append(img)
#     image_id = canvas.create_image(640, 360, image=img)
#     image_ids.append(image_id)
# except tk.TclError as e:
#     print(f"이미지 로딩 오류: {e}")
#     print(f"시도한 경로: ../img/background_pages/team_input.png")
#     img = tk.PhotoImage(width=1, height=1)

# canvas.bind("<Button-1>", handle_page)
# root.bind("<Key>", handle_input)

# root.mainloop()